In [2]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
#pip3 install bpmll
#using bpmll in the loss
from bpmll import bp_mll_loss
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
#pip install --upgrade gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
train_data = json.load(open('train.json'))

In [4]:
#take the author out 
label_au = []
for i,d in enumerate(train_data):
    author = np.array(d['authors'])
    author_100 = author[author<100]
    #length is 101, put the author number which greater than 100 in the last one in the array 
    au_range = np.zeros(101)
    if len(author_100) == 0:
        au_range[-1] = 1 
    else:
        au_range[author_100] = 1
    label_au.append(au_range)

In [5]:
author_label_train = np.array(label_au)

In [6]:
# using word2vec
title_l = []
abstract_l = []
for data in train_data:
    title_l.append(data['title'])
    abstract_l.append(data['abstract'])
title_abst = title_l+abstract_l
docum = [TaggedDocument(docm,[i]) for i , docm in enumerate(title_abst)]

In [7]:
model_word = Doc2Vec(docum,vector_size = 256,window = 2,min_count =1)

In [8]:
model_word.infer_vector(np.asarray(title_abst[0],dtype='str'))

array([ 1.22682028e-03, -1.19128090e-03, -1.24879624e-03, -4.72046435e-04,
        7.03190686e-04,  1.93640031e-03,  1.08795310e-03,  1.35309761e-03,
        3.93903349e-04,  5.04254596e-04,  1.41281774e-03, -2.56883213e-05,
        1.28149311e-03,  4.29248903e-05,  1.70569820e-03,  1.42676290e-05,
        1.88375614e-03, -7.70724495e-04,  8.41811998e-04,  1.94717525e-03,
       -1.54935766e-03,  5.49636548e-04, -3.23833083e-04, -8.55718739e-04,
       -1.91921019e-03,  3.82552622e-04, -8.73192330e-04, -1.03918510e-03,
       -1.70134462e-03, -1.58110936e-03,  6.11537136e-04, -1.82162481e-03,
       -1.75517623e-03,  1.02145458e-03, -1.18117128e-03, -1.14864856e-03,
        2.87811970e-04, -1.69097004e-03, -1.74781273e-03, -7.17488583e-05,
        1.50962314e-03, -1.52260275e-03,  5.03086485e-04,  1.01283239e-03,
       -1.55485072e-03, -5.10125305e-04, -1.57363061e-03, -8.71573226e-04,
       -1.01270224e-03, -1.61302625e-04,  1.10509433e-03, -1.31899258e-04,
        1.35625387e-03,  

In [9]:
#doc2vec version
vect_ta = []
for cont in train_data:
    titl = cont['title']
    abstr = cont['abstract']
    all_ta = titl+abstr
    vector = np.zeros(256)
    for wor in all_ta:
        vector +=model_word[wor]
    vector = vector/len(all_ta)
    vect_ta.append(vector)
vect_ta = np.array(vect_ta)

In [10]:
#using one hot method 
venue_l = []
for cont in train_data:
    #venue id from 0 to 464, include ' ' will have 466 id
    vn_n = np.zeros(466)
    ven = cont['venue']
    if ven == '':
        vn_n[-1] = 1
    else:
        vn_n[ven] = 1
    venue_l.append(vn_n)
venue_l = np.array(venue_l)

In [11]:
#find out of distribution 
#it means that find out the id more than 100 but corparate with less than id's is 100
co_auth = {}
for cont in train_data:
    auth = np.array(cont['authors'])
    auth_100 = auth[auth<100]
    auth_101 = auth[auth>=100]
    if len(auth_100)!=0:
        for j in auth_101:
            co_auth[j] = 1


In [13]:
#using one hot vector method for co-author
co_author = []
idx_coau ={}
coau_l = list(co_auth.keys())
for d,idx in enumerate(coau_l):
    idx_coau[idx] = d
    

for conte in train_data:
    auth = np.array(cont['authors'])
    auth_100 = auth[auth<100]
    auth_101 = auth[auth>=100]
    co_n = np.zeros(6576)# len of co-author
    if len(auth_100) == 0:
        co_n[-1] = 1
    else:
        for i in auth_101:
            co_n[idx_coau[i]] =1
    co_author.append(co_n)
co_author = np.array(co_author)

In [14]:
# adding the array together 
thr_feat = np.concatenate((vect_ta,venue_l),axis =1)

In [15]:
mix_featr = np.concatenate((thr_feat ,co_author),axis =1)

In [16]:
np.save('train_doc.npy',mix_featr)

In [17]:
test_data =json.load(open('test.json'))

In [18]:
#doc2vec version
vect_test = []
for cont in test_data:
    titl = cont['title']
    abstr = cont['abstract']
    all_ta = titl+abstr
    vector = np.zeros(256)
    for wor in all_ta:
        vector +=model_word[wor]
    vector = vector/len(all_ta)
    vect_test.append(vector)
vect_test = np.array(vect_test)

In [19]:
#using one hot method 
venue_test = []
for cont in test_data:
    #venue id from 0 to 464, include ' ' will have 466 id
    vn_n = np.zeros(466)
    ven = cont['venue']
    if ven == '':
        vn_n[-1] = 1
    else:
        vn_n[ven] = 1
    venue_test.append(vn_n)
venue_test = np.array(venue_test)

In [20]:
co_author_test = []
for conte in test_data:
    auth = np.array(cont['coauthors'])
    auth_100 = auth[auth<100]
    auth_101 = auth[auth>=100]
    co_n = np.zeros(6576)# len of co-author
    if len(auth_100) == 0:
        co_n[-1] = 1
    else:
        for i in auth_101:
            co_n[idx_coau[i]] = 1
    co_author_test.append(co_n)
co_author_test = np.array(co_author_test)

In [21]:
thr_feat_test = np.concatenate((vect_test,venue_test),axis =1)

In [22]:
mix_featr_test = np.concatenate((thr_feat_test ,co_author_test),axis =1)

In [23]:
np.save('test_word.npy',mix_featr_test)
#save the feature data 

In [24]:
# classification model
#using wor2vec to test
x =np.load('train_doc.npy')

In [25]:
st_sc = StandardScaler()
x = st_sc.fit_transform(x)

In [26]:
# if want to remove the some of the auther id which include after 100
idx_mo100 = np.where(author_label_train[:,-1] == 1)[0]
idx_un100 = np.where(author_label_train[:,-1] != 1)[0]

In [27]:
un_num = x[idx_un100]
mo_num = x[idx_mo100]
un_l = author_label_train[idx_un100]
mo_l = author_label_train[idx_mo100]


In [28]:
# we need add some dataset that make the total number of dataset is the close to orginal
repeat_time = int((mo_num.shape[0] - un_num.shape[0])/un_num.shape[0])
repeat_un = np.repeat(un_num,2,axis = 0)

In [29]:
repeat_label = np.repeat(un_l,2,axis = 0)

In [30]:
x = np.concatenate((x,repeat_un))
label_repeat = np.concatenate((author_label_train,repeat_label))

In [31]:
#after add dataset 
x_train,x_test,y_train,y_test = train_test_split(x,label_repeat,test_size = 0.3,random_state =100)

In [32]:
learning_rate = PolynomialDecay(initial_learning_rate = 0.001,decay_steps = 150,end_learning_rate = 0.00025,power = 0.5)
model_all = Sequential()
model_all.add(Dense(2048, input_dim = x_train.shape[1],activation= 'tanh'))
model_all.add(Dropout(0.3))
model_all.add(Dense(512,activation ='tanh'))
model_all.add(Dropout(0.3))
model_all.add(Dense(512,activation ='tanh'))
model_all.add(Dropout(0.3))
model_all.add(Dense(256,activation ='tanh'))
model_all.add(Dropout(0.3))
model_all.add(Dense(y_train.shape[1],activation ='sigmoid'))
model_all.compile(loss = 'binary_crossentropy',optimizer=Adam(learning_rate))
model_all.fit(x_train,y_train,epochs = 150,batch_size = 128)

2022-10-14 20:22:42.206281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/150
223/223 [==============================] - 20s 85ms/step - loss: 0.1414
Epoch 2/150
223/223 [==============================] - 19s 85ms/step - loss: 0.0485
Epoch 3/150
223/223 [==============================] - 21s 93ms/step - loss: 0.0464
Epoch 4/150
223/223 [==============================] - 21s 96ms/step - loss: 0.0452
Epoch 5/150
223/223 [==============================] - 21s 92ms/step - loss: 0.0443
Epoch 6/150
223/223 [==============================] - 20s 91ms/step - loss: 0.0433
Epoch 7/150
223/223 [==============================] - 20s 92ms/step - loss: 0.0422
Epoch 8/150
223/223 [==============================] - 21s 93ms/step - loss: 0.0431
Epoch 9/150
223/223 [==============================] - 21s 93ms/step - loss: 0.0407
Epoch 10/150
223/223 [==============================] - 20s 92ms/step - loss: 0.0396
Epoch 11/150
223/223 [==============================] - 21s 93ms/step - loss: 0.0388
Epoch 12/150
223/223 [==============================] - 20s 92ms/step - lo

223/223 [==============================] - 20s 90ms/step - loss: 0.0105
Epoch 98/150
223/223 [==============================] - 20s 89ms/step - loss: 0.0103
Epoch 99/150
223/223 [==============================] - 20s 90ms/step - loss: 0.0105
Epoch 100/150
223/223 [==============================] - 20s 89ms/step - loss: 0.0102
Epoch 101/150
223/223 [==============================] - 20s 90ms/step - loss: 0.0101
Epoch 102/150
223/223 [==============================] - 20s 90ms/step - loss: 0.0098
Epoch 103/150
223/223 [==============================] - 20s 90ms/step - loss: 0.0099
Epoch 104/150
223/223 [==============================] - 20s 90ms/step - loss: 0.0096
Epoch 105/150
223/223 [==============================] - 21s 92ms/step - loss: 0.0095
Epoch 106/150
223/223 [==============================] - 20s 91ms/step - loss: 0.0094
Epoch 107/150
223/223 [==============================] - 20s 92ms/step - loss: 0.0095
Epoch 108/150
223/223 [==============================] - 21s 93ms/step

In [33]:
predi_re = model_all.predict(x_train)

891/891 [==============================] - 9s 10ms/step


In [34]:
predi_re = np.where(predi_re >=0.5,1,0)

In [35]:
f1_score(y_true = y_train, y_pred = predi_re,average ='samples')

0.9962193905139272

In [36]:
predi_test = model_all.predict(x_test)
predi_test = np.where(predi_test >=0.5,1,0)
f1_score(y_true = y_test, y_pred = predi_test,average ='samples')

382/382 [==============================] - 4s 10ms/step


0.8625314681722602

In [37]:
y = np.load('test_word.npy')

In [38]:
y = st_sc.transform(y)

In [39]:
predic_final = model_all.predict(y)

25/25 [==============================] - 0s 10ms/step


In [40]:
predic_final_idx = [np.where(i >=0.5) for i in predic_final]

In [41]:
csv_t = pd.DataFrame(predic_final_idx, columns =['Predict'])

In [42]:
csv_t

,Predict
0,[100]
1,[100]
2,[100]
3,[100]
4,[100]
...,...
795,[100]
796,[]
797,[100]
798,[100]


In [ ]:
csv_t.to_csv('predict.csv')